## Introduction
This notebook implements an ASD (Autism Spectrum Disorder) prediction model using the Xception model. The dataset is organized into `train`, `valid`, and `test` folders, each with subfolders `Autistics` and `Non_Autistics`. The goal is to achieve ~91% accuracy, as reported in the study.

In [4]:
# Import necessary libraries
import tensorflow as tf
from tensorflow.keras.applications import Xception
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.optimizers import Adam
import numpy as np
import os

## Set Random Seed
We set a random seed to ensure reproducibility of results.

In [5]:
# Set random seed for reproducibility
tf.random.set_seed(42)
np.random.seed(42)

## Define Dataset Paths
Specify the paths to the dataset folders. Update `base_dir` to match the location of your dataset on your computer.

In [6]:
# Define paths to dataset
base_dir = 'data'  # Update this to your dataset path
train_dir = os.path.join(base_dir, 'train')
valid_dir = os.path.join(base_dir, 'valid')
test_dir = os.path.join(base_dir, 'test')

## Set Image Parameters
Define the image size, batch size, and number of classes for the model.

In [7]:
# Image parameters
IMG_SIZE = (299, 299)  # Xception default input size
BATCH_SIZE = 32
NUM_CLASSES = 2  # Autistics vs. Non_Autistics

## Data Augmentation and Preprocessing
Set up data generators to preprocess and augment the training images, and preprocess the validation and test images.

In [8]:
# Data augmentation and preprocessing for training
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Preprocessing for validation and test (no augmentation)
valid_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

## Load the Data
Load the images from the `train`, `valid`, and `test` folders using the data generators.

In [9]:
# Load and preprocess training, validation, and testing data
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    shuffle=True
)

valid_generator = valid_datagen.flow_from_directory(
    valid_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    shuffle=False
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    shuffle=False
)

Found 2540 images belonging to 2 classes.
Found 100 images belonging to 2 classes.
Found 300 images belonging to 2 classes.


## Build the Model
Load the Xception model, freeze its layers, and add custom layers for ASD prediction.

In [10]:
# Load Xception model with pre-trained ImageNet weights
base_model = Xception(weights='imagenet', include_top=False, input_shape=(299, 299, 3))

# Freeze the base model layers
base_model.trainable = False

# Add custom layers on top
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(1, activation='sigmoid')(x)

# Create the final model
model = Model(inputs=base_model.input, outputs=predictions)

## Compile the Model
Compile the model with the Adam optimizer, binary crossentropy loss, and accuracy metric.

In [11]:
# Compile the model
model.compile(optimizer=Adam(learning_rate=0.0003),
              loss='binary_crossentropy',
              metrics=['accuracy'])

## Model Summary
Display the architecture of the model.

In [ ]:
# Model summary
model.summary()

## Train the Model
Train the model for 20 epochs using the training data, with validation on the validation data.

In [ ]:
# Train the model
EPOCHS = 20
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // BATCH_SIZE,
    epochs=EPOCHS,
    validation_data=valid_generator,
    validation_steps=valid_generator.samples // BATCH_SIZE
)

C:\Users\Dell\AppData\Roaming\Python\Python312\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/20
79/79 ━━━━━━━━━━━━━━━━━━━━ 282s 3s/step - accuracy: 0.5907 - loss: 0.6640 - val_accuracy: 0.7396 - val_loss: 0.5784
Epoch 2/20
 1/79 ━━━━━━━━━━━━━━━━━━━━ 3:55 3s/step - accuracy: 0.7188 - loss: 0.5628

C:\Users\Dell\AppData\Roaming\Python\Python312\site-packages\keras\src\trainers\epoch_iterator.py:107: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


79/79 ━━━━━━━━━━━━━━━━━━━━ 13s 127ms/step - accuracy: 0.7188 - loss: 0.5628 - val_accuracy: 0.7396 - val_loss: 0.5782
Epoch 3/20
28/79 ━━━━━━━━━━━━━━━━━━━━ 3:00 4s/step - accuracy: 0.7106 - loss: 0.5752

## Evaluate the Model
Evaluate the model on the test data to check its accuracy.

In [ ]:
# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(test_generator)
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")

## Save the Model
Save the trained model to a file for future use.

In [ ]:
# Save the model
model.save('xception_asd_model.h5')

## Load the Saved Model
Load the saved Xception model from the `xception_asd_model.h5` file to use it for predictions without retraining. 

In [ ]:
# Load the saved model
from tensorflow.keras.models import load_model

# Path to the saved model
model_path = 'xception_asd_model.h5'  # Update this if the model is in a different directory

# Load the model
loaded_model = load_model(model_path)
print("Model loaded successfully!")

## Define Prediction Function
Define a function to predict ASD from a single image.

In [ ]:
# Function to predict ASD from a single image
def predict_asd(image_path, model):
    from tensorflow.keras.preprocessing import image
    img = image.load_img(image_path, target_size=IMG_SIZE)
    img_array = image.img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)
    prediction = model.predict(img_array)[0][0]
    label = 'Autistic' if prediction > 0.5 else 'Non_Autistic'
    confidence = prediction if prediction > 0.5 else 1 - prediction
    return label, confidence

## Make a Prediction
Use the prediction function to classify a single image. Update the `image_path` to point to an actual image in your dataset.

In [ ]:
# Example prediction
image_path = './data/test/Autistic/Autistic.7.jpg'  # Update this to a real image path
label, confidence = predict_asd(image_path, model)
print(f"Prediction: {label}, Confidence: {confidence:.2f}")

## Explainable AI with Grad-CAM
To understand which parts of the image the model focuses on when making its prediction, we use Grad-CAM (Gradient-weighted Class Activation Mapping). This technique generates a heatmap highlighting the regions of the image that contributed most to the model's decision (e.g., `Autistic` or `Non_Autistic`).

In [ ]:
# Install and import libraries for visualization
import matplotlib.pyplot as plt
import cv2  # For image processing and heatmap overlay
import numpy as np

## Define Grad-CAM Function
We define a function to compute the Grad-CAM heatmap for a given image and model. This function identifies the last convolutional layer in the Xception model, computes the gradients, and generates the heatmap.

In [ ]:
def get_gradcam_heatmap(model, img_path, img_size=(299, 299), last_conv_layer_name="block14_sepconv2_act"):
    """
    Generate a Grad-CAM heatmap for the given image and model.
    
    Args:
        model: The trained model (Xception in this case).
        img_path: Path to the image.
        img_size: Size to resize the image (default: 299x299 for Xception).
        last_conv_layer_name: Name of the last convolutional layer in the model.
    
    Returns:
        heatmap: The Grad-CAM heatmap.
        img: The original image.
    """
    # Load and preprocess the image
    img = tf.keras.preprocessing.image.load_img(img_path, target_size=img_size)
    img_array = tf.keras.preprocessing.image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    img_array = img_array / 255.0  # Normalize to [0, 1]
    
    # Convert img_array to a TensorFlow tensor
    img_tensor = tf.convert_to_tensor(img_array, dtype=tf.float32)

    # Get the last convolutional layer
    last_conv_layer = model.get_layer(last_conv_layer_name)
    
    # Create a submodel that outputs the last conv layer and the final predictions
    grad_model = tf.keras.models.Model(
        [model.inputs], [last_conv_layer.output, model.output]
    )

    # Compute gradients of the predicted class with respect to the last conv layer
    with tf.GradientTape() as tape:
        # Watch the input tensor
        tape.watch(img_tensor)
        # Pass the input as a list to match model.inputs structure
        conv_outputs, predictions = grad_model([img_tensor])
        # For binary classification, use the prediction directly
        predicted_class = tf.cast(predictions > 0.5, "float32")
        loss = predictions[:, 0] if predicted_class[0] > 0.5 else 1 - predictions[:, 0]

    # Get the gradients
    grads = tape.gradient(loss, conv_outputs)[0]
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1))  # Global average pooling

    # Multiply each channel in the feature map by its gradient importance
    conv_outputs = conv_outputs[0]  # Remove batch dimension
    heatmap = tf.reduce_mean(tf.multiply(conv_outputs, pooled_grads), axis=-1)
    heatmap = np.maximum(heatmap, 0)  # ReLU to keep only positive contributions
    heatmap = heatmap / np.max(heatmap)  # Normalize to [0, 1]

    # Resize heatmap to match the original image size
    heatmap = cv2.resize(heatmap, (img_size[1], img_size[0]))
    heatmap = np.uint8(255 * heatmap)  # Convert to 0-255 range for visualization

    return heatmap, img

## Visualize the Grad-CAM Heatmap
We overlay the Grad-CAM heatmap on the original image to see which regions the model focused on when making its prediction.

In [ ]:
# Use the same image path as in the prediction
# Ensure this matches the image_path you used in the prediction cell
image_path = './Autistic.62.jpg'  # Replace with your actual image path

# Get the Grad-CAM heatmap
heatmap, original_img = get_gradcam_heatmap(model, image_path, img_size=IMG_SIZE)

# Convert the heatmap to RGB for overlay
heatmap = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET)  # Apply JET colormap (blue to red)
heatmap = cv2.cvtColor(heatmap, cv2.COLOR_BGR2RGB)  # Convert BGR to RGB for matplotlib

# Convert original image to numpy array for overlay
original_img = tf.keras.preprocessing.image.img_to_array(original_img)

# Resize heatmap to match the original image dimensions
heatmap = np.float32(heatmap) / 255
alpha = 0.4  # Transparency factor for the heatmap
overlay = (1 - alpha) * original_img + alpha * heatmap * 255  # Overlay the heatmap
overlay = np.clip(overlay, 0, 255).astype(np.uint8)  # Ensure values are in valid range

# Display the original image, heatmap, and overlay
plt.figure(figsize=(15, 5))

# Original image
plt.subplot(1, 3, 1)
plt.imshow(original_img.astype(np.uint8))
plt.title("Original Image")
plt.axis('off')

# Heatmap
plt.subplot(1, 3, 2)
plt.imshow(heatmap)
plt.title("Grad-CAM Heatmap")
plt.axis('off')

# Overlay
plt.subplot(1, 3, 3)
plt.imshow(overlay)
plt.title("Overlay (Original + Heatmap)")
plt.axis('off')

plt.show()

# Print the prediction again for reference
label, confidence = predict_asd(image_path, model)
print(f"Prediction: {label}, Confidence: {confidence:.2f}")

## List Test Images and Run Grad-CAM
We will list a few images from the `test` dataset (from both `Autistics` and `Non_Autistics` subfolders) and run Grad-CAM to visualize the regions of the images that the model focuses on when making its predictions.

In [ ]:
import os

# Define the path to the test dataset
test_dir = 'data/test'  # Update this to your test dataset path

# Get paths to images in the Autistics and Non_Autistics subfolders
autistic_dir = os.path.join(test_dir, 'Autistic')
non_autistic_dir = os.path.join(test_dir, 'Non_Autistic')

# List a few images from each subfolder (e.g., first 3 images)
autistic_images = [os.path.join(autistic_dir, fname) for fname in os.listdir(autistic_dir)[45:52] if fname.endswith(('.jpg', '.jpeg', '.png'))]
non_autistic_images = [os.path.join(non_autistic_dir, fname) for fname in os.listdir(non_autistic_dir)[45:52] if fname.endswith(('.jpg', '.jpeg', '.png'))]

# Combine the lists
test_images = autistic_images + non_autistic_images

# Print the selected images
print("Selected test images:")
for img_path in test_images:
    print(img_path)

## Run Grad-CAM on Test Images
We will run Grad-CAM on the selected test images and display the original image, Grad-CAM heatmap, and overlay for each image, along with the model's prediction.

In [ ]:
# Ensure the get_gradcam_heatmap and predict_asd functions are defined (they should already be in earlier cells)
# Also ensure matplotlib, cv2, and numpy are imported (from the Grad-CAM setup)

# Run Grad-CAM on each test image
for img_path in test_images:
    # Get the Grad-CAM heatmap
    heatmap, original_img = get_gradcam_heatmap(model, img_path, img_size=IMG_SIZE)

    # Convert the heatmap to RGB for overlay
    heatmap_rgb = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET)  # Apply JET colormap (blue to red)
    heatmap_rgb = cv2.cvtColor(heatmap_rgb, cv2.COLOR_BGR2RGB)  # Convert BGR to RGB for matplotlib

    # Convert original image to numpy array for overlay
    original_img = tf.keras.preprocessing.image.img_to_array(original_img)

    # Resize heatmap to match the original image dimensions
    heatmap_rgb = np.float32(heatmap_rgb) / 255
    alpha = 0.4  # Transparency factor for the heatmap
    overlay = (1 - alpha) * original_img + alpha * heatmap_rgb * 255  # Overlay the heatmap
    overlay = np.clip(overlay, 0, 255).astype(np.uint8)  # Ensure values are in valid range

    # Display the original image, heatmap, and overlay
    plt.figure(figsize=(15, 5))

    # Original image
    plt.subplot(1, 3, 1)
    plt.imshow(original_img.astype(np.uint8))
    plt.title("Original Image")
    plt.axis('off')

    # Heatmap
    plt.subplot(1, 3, 2)
    plt.imshow(heatmap_rgb)
    plt.title("Grad-CAM Heatmap")
    plt.axis('off')

    # Overlay
    plt.subplot(1, 3, 3)
    plt.imshow(overlay)
    plt.title("Overlay (Original + Heatmap)")
    plt.axis('off')

    # Add a super title with the image path
    plt.suptitle(f"Image: {os.path.basename(img_path)}", fontsize=16)
    plt.show()

    # Print the prediction
    label, confidence = predict_asd(img_path, model)
    print(f"Prediction for {os.path.basename(img_path)}: {label}, Confidence: {confidence:.2f}\n")

## Evaluate Predictions on Selected Test Images
We will compare the model's predictions with the true labels for the selected test images to calculate the accuracy on this subset and understand why the predictions might be incorrect.

In [ ]:
# Function to get the true label from the image path
def get_true_label(img_path):
    if 'Autistics' in img_path:
        return 'Autistic'
    elif 'Non_Autistics' in img_path:
        return 'Non_Autistic'
    else:
        return None

# Evaluate predictions on the selected test images
correct_predictions = 0
total_images = len(test_images)

print("Evaluating predictions on selected test images:")
for img_path in test_images:
    # Get the true label
    true_label = get_true_label(img_path)
    
    # Get the predicted label
    predicted_label, confidence = predict_asd(img_path, model)
    
    # Check if the prediction is correct
    is_correct = (true_label == predicted_label)
    if is_correct:
        correct_predictions += 1
    
    # Print the result
    print(f"Image: {os.path.basename(img_path)}")
    print(f"True Label: {true_label}")
    print(f"Predicted Label: {predicted_label}, Confidence: {confidence:.2f}")
    print(f"Correct: {is_correct}\n")

# Calculate accuracy on this subset
subset_accuracy = (correct_predictions / total_images) * 100
print(f"Accuracy on selected test images: {subset_accuracy:.2f}%")
print(f"Number of correct predictions: {correct_predictions} out of {total_images}")